In [ ]:
!pip install transformers
!pip install torch
!pip install accelerate

In [91]:
# pip install accelerate
from transformers import AutoTokenizer, OPTForCausalLM, StoppingCriteria
import torch

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("facebook/galactica-6.7b")
model = OPTForCausalLM.from_pretrained("facebook/galactica-6.7b", device_map="auto")

In [71]:
stop_words_ids = [
    tokenizer(stop_word, return_tensors='pt')['input_ids'].squeeze() for stop_word in ['\n']]

class StoppingCriteriaSub(StoppingCriteria):
    def __init__(self, stops = []):
      super().__init__()
      self.stops = stops

    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor):
      for stop in self.stops:
          if stop == input_ids[0][-1]:
              print(stop)
              print(input_ids)
              return True
      return False

stopping_criteria = StoppingCriteriaSub(stop_words_ids)
print(stop_words_ids)

[tensor(221)]


In [19]:
input_text = "The Transformer architecture [START_REF]"
input_ids = tokenizer(input_text, return_tensors="pt").input_ids

outputs = model.generate(input_ids, max_new_tokens=20, stopping_criteria=[stopping_criteria])
print(tokenizer.decode(outputs[0]))

The Transformer architecture [START_REF] Attention is All you Need, Vaswani[END_REF] is a sequence-to-sequence model that uses self


In [74]:
input_text = "The Figure/Table shows the relationship between population and average income.\n\
The caption of the figure mentioned in the above sentence should be\n\n"
input_ids = tokenizer(input_text, return_tensors="pt").input_ids

outputs = model.generate(input_ids, max_new_tokens=20, stopping_criteria=[stopping_criteria])
# outputs = model.generate(input_ids, max_new_tokens=20)
# print(outputs[0])
print(tokenizer.decode(outputs[0]))

tensor(221)
tensor([[  592,  1482,    37,  2044,  1586,   286,  2191,   658,  1918,   312,
          1908, 10011,    36,   221,   592, 33223,   299,   286,  4374,  4789,
           301,   286,  1504,  9553,  1502,   391,   221,   221,  1095,   243,
            39,    48, 10862,   658,  1918,   312,  1908, 10011,    36,   221]])
The Figure/Table shows the relationship between population and average income.
The caption of the figure mentioned in the above sentence should be

Figure 1: Relationship between population and average income.



In [88]:
eos_token_id = int(tokenizer('\n', return_tensors="pt").input_ids[0][0])
input_text = "The Figure/Table shows the relationship between population and average income.\n\
The caption of the figure mentioned in the above sentence should be"
input_ids = tokenizer(input_text, return_tensors="pt").input_ids

outputs = model.generate(input_ids, min_new_tokens=5, max_new_tokens=20, eos_token_id=eos_token_id)
# outputs = model.generate(input_ids, max_new_tokens=20)
print(outputs[0])
print(tokenizer.decode(outputs[0]))

tensor([  592,  1482,    37,  2044,  1586,   286,  2191,   658,  1918,   312,
         1908, 10011,    36,   221,   592, 33223,   299,   286,  4374,  4789,
          301,   286,  1504,  9553,  1502,   391,    48,   243,    24,   592,
         1482,    37,  2044,  1586,   286,  2191,   658,  1918,   312,  1908,
        10011,    36,    24,   221])
The Figure/Table shows the relationship between population and average income.
The caption of the figure mentioned in the above sentence should be: "The Figure/Table shows the relationship between population and average income."

